In [2]:
exec(open("init_notebook.py").read())

current working dirc:\muktadir\junction-art


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import dill
import os
import numpy as np
from junctionart.library.Configuration import Configuration
from junctionart.junctions.LaneConfigurationStrategies import LaneConfigurationStrategies
from junctionart.junctions.IntersectionValidator import IntersectionValidator
from junctionart.junctions.SequentialJunctionBuilder import SequentialJunctionBuilder
import junctionart.extensions as extensions
import pyodrx as pyodrx
import logging
from tqdm import tqdm
logging.basicConfig(level=logging.INFO)

In [43]:
class SequentialIntersectionExporter():

    def __init__(self):
        
        self.configuration = Configuration()
        self.outputDir= os.path.join(os.getcwd(), 'output')
        self.lastId = 0
        self.seed = 2
        self.builder = SequentialJunctionBuilder(
                                                    minAngle=np.pi/10, 
                                                    maxAngle=np.pi * .75,
                                                    straightRoadLen=5, 
                                                    probLongConnection=0.5,
                                                    probMinAngle=0.5,
                                                    probRestrictedLane=0.2,
                                                    maxConnectionLength=50,
                                                    minConnectionLength=20,
                                                    random_seed=self.seed)
        
        self.randomState =self.configuration.get("random_state")
        self.validator = IntersectionValidator()

        pass
    
    def genId(self):
        self.lastId += 1
        return self.lastId
    
    def gen(self,
        nLegs = 3,
        minLanePerSide = 1,
        maxLanePerSide = 2,
        numberOfIntersections = 10):

        created = 0
        intersections = []
        # create 
        with tqdm(total=numberOfIntersections, desc=f"{nLegs}way intersections {minLanePerSide}, {maxLanePerSide}") as pbar:
            while created < numberOfIntersections:
                path = self.configuration.get("harvested_straight_roads")
                intersection = self.builder.createWithRandomLaneConfigurations(path, 
                                    self.genId(), 
                                    maxNumberOfRoadsPerJunction=nLegs, 
                                    maxLanePerSide=maxLanePerSide, 
                                    minLanePerSide=minLanePerSide, 
                                    internalConnections=True, 
                                    cp1=pyodrx.ContactPoint.end,
                                    internalLinkStrategy = LaneConfigurationStrategies.SPLIT_ANY,
                                    getAsOdr=False)


#                 odr = intersection.odr
                # xmlPath = f"output/test_createWithRandomLaneConfigurations-split-any-{maxNumberOfRoadsPerJunction}-{sl}.xodr"
#                 xmlPath = f"output/seed-{self.seed}-{maxNumberOfRoadsPerJunction}-way-{sl}.xodr"
#                 odr.write_xml(xmlPath)
                isValid = self.validator.validateIncidentPoints(intersection, self.builder.minConnectionLength)
                if isValid and len(intersection.incidentRoads) == nLegs:
                    intersections.append(intersection)
                    created += 1
                    pbar.update(1)

#                 if created % 1000 == 0:
#                     print(f"generated {created}")
        
        
        return intersections
    
    def save(self, outputPath, intersections):
        with open(outputPath, 'wb') as handler:
            print(f"Writing out {len(intersections)} intersections at {outputPath}")
            dill.dump(intersections, handler)
    
    def genNSave(self, 
        nLegs = 3,
        minLanePerSide = 1,
        maxLanePerSide = 2,
        numberOfIntersections = 10):
        
        intersections = self.gen(
            nLegs = nLegs,
            minLanePerSide = minLanePerSide,
            maxLanePerSide = maxLanePerSide,
            numberOfIntersections = numberOfIntersections
        )
        
        laneStr = ""
        if minLanePerSide == maxLanePerSide and minLanePerSide == 1:
            laneStr = "-2lane"
        
        outputPath = os.path.join(self.outputDir, f"Sequential-intersections{laneStr}-{nLegs}way-{numberOfIntersections}.dill")

        self.save(outputPath, intersections)
        
        

                    


In [44]:
exporter = SequentialIntersectionExporter()

In [45]:
for nLegs in range(3, 7):
    print(f"Generating {nLegs}ways")
    exporter.genNSave(
            nLegs = nLegs,
            minLanePerSide = 1,
            maxLanePerSide = 2,
            numberOfIntersections = 1000
    )
    exporter.genNSave(
            nLegs = nLegs,
            minLanePerSide = 1,
            maxLanePerSide = 1,
            numberOfIntersections = 1000
    )

Generating 3ways


3way intersections 1, 2: 100%|█████████████████████████████████████████████████████| 1000/1000 [00:32<00:00, 30.80it/s]


Writing out 1000 intersections at c:\muktadir\junction-art\output\Sequential-intersections-3way-1000.dill


3way intersections 1, 1: 100%|█████████████████████████████████████████████████████| 1000/1000 [00:24<00:00, 40.20it/s]


Writing out 1000 intersections at c:\muktadir\junction-art\output\Sequential-intersections-2lane-3way-1000.dill
Generating 4ways


...

Writing out 1000 intersections at c:\muktadir\junction-art\output\Sequential-intersections-2lane-5way-1000.dill
Generating 6ways


6way intersections 1, 2: 100%|█████████████████████████████████████████████████████| 1000/1000 [03:19<00:00,  5.02it/s]


Writing out 1000 intersections at c:\muktadir\junction-art\output\Sequential-intersections-6way-1000.dill


6way intersections 1, 1: 100%|█████████████████████████████████████████████████████| 1000/1000 [02:09<00:00,  7.73it/s]


Writing out 1000 intersections at c:\muktadir\junction-art\output\Sequential-intersections-2lane-6way-1000.dill
